In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# CONFIG
# -----------------------------
RUNTIME_FOLDER = Path("data/runtime")
OUTPUT_FOLDER = Path("output/hourly")
OUTPUT_TOPSCORE = Path("output/topscore")
OUTPUT_TOPHEATMAP = Path("output/topheatmap")
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)
OUTPUT_TOPSCORE.mkdir(parents=True, exist_ok=True)
OUTPUT_TOPHEATMAP.mkdir(parents=True, exist_ok=True)

LOOKBACK_MINUTES = 60

# -----------------------------
# STEP 1 — Load recent files
# -----------------------------
now = datetime.now()
cutoff = now - timedelta(minutes=LOOKBACK_MINUTES)

csv_files = sorted(RUNTIME_FOLDER.glob("runtime_suggestions_*.csv"))
recent_files = []
for f in csv_files:
    try:
        ts = datetime.strptime(f.stem.split("_")[-1], "%Y%m%d%H%M%S")
        if ts >= cutoff:
            recent_files.append(f)
    except:
        continue

if not recent_files:
    raise RuntimeError("❌ No recent runtime_suggestions files found.")

# Load all recent files
df_list = []
for f in recent_files:
    ts = datetime.strptime(f.stem.split("_")[-1], "%Y%m%d%H%M%S")
    temp = pd.read_csv(f)
    temp["timestamp"] = ts
    df_list.append(temp)

df = pd.concat(df_list, ignore_index=True)
df.sort_values("timestamp", inplace=True)

# -----------------------------
# STEP 2 — Precompute metrics
# -----------------------------
# Aggregated stats per symbol
agg_cols = ["RuntimeScore", "HotScore_today", "TrendScore", "regularMarketPrice"]
grouped = df.groupby("symbol")[agg_cols].mean().sort_values("RuntimeScore", ascending=False)

# Appearance counts
appearance_counts = df["symbol"].value_counts()

# Rolling metrics per timestamp
rolling_metrics = df.groupby("timestamp")[["TrendScore", "RuntimeScore"]].mean().reset_index()
# -----------------------------
# STEP 3 — Generate charts
# -----------------------------
timestamp_str = datetime.now().strftime("%Y%m%d%H%M%S")

# 1️⃣ Heatmap Scores
# 1️⃣ Heatmap Scores — improved readability
heatmap_data = df.pivot_table(index="symbol", columns="timestamp", values="RuntimeScore", fill_value=0)

plt.figure(figsize=(14, max(8, len(heatmap_data)/2)))  # increase height dynamically
sns.heatmap(heatmap_data, cmap="viridis", annot=False, cbar_kws={'label': 'RuntimeScore'})

plt.title("Heatmap: RuntimeScore Over Time")
plt.xlabel("Timestamp")
plt.ylabel("Symbol")

# Rotate x labels for clarity
plt.xticks(rotation=45, ha="right", fontsize=8)
plt.yticks(fontsize=8)

# Optional: show fewer x-ticks if too dense
for i, label in enumerate(plt.gca().get_xticklabels()):
    if i % 2 != 0:  # show every 2nd label
        label.set_visible(False)

plt.tight_layout()
plt.savefig(OUTPUT_TOPHEATMAP / f"heatmap_score_{timestamp_str}.png", dpi=200)
plt.close()


# 2️⃣ Top RuntimeScore
plt.figure(figsize=(10,6))
top_runtime = grouped.head(15).sort_values("RuntimeScore", ascending=True)
colors = sns.color_palette("coolwarm", n_colors=len(top_runtime))
plt.bar(top_runtime.index, top_runtime["RuntimeScore"], color=colors)
plt.title("Top RuntimeScore (Average)")
plt.xlabel("RuntimeScore")
plt.tight_layout()
plt.savefig(OUTPUT_TOPSCORE / f"top_score_{timestamp_str}.png", dpi=200)
plt.close()

# 3️⃣ Appearance Counts
plt.figure(figsize=(10,6))
top_appearances = appearance_counts.head(15).sort_values(ascending=True)
colors = sns.color_palette("mako", n_colors=len(top_appearances))
plt.bar(top_appearances.index, top_appearances.values, color=colors)
plt.title("Top Symbols by Appearance Count")
plt.xlabel("Appearances")
plt.tight_layout()
plt.savefig(OUTPUT_FOLDER / f"appearance_counts_{timestamp_str}.png", dpi=200)
plt.close()


print(f"✅ Charts saved under {OUTPUT_FOLDER} with timestamp {timestamp_str}")


✅ Charts saved under output\hourly with timestamp 20260211112956
